In [1]:
import gradio as gr
from pydub import AudioSegment
from utilities import get_available_languages, process_audio_lyric_extraction, process_audio_lyric_translation

c:\Users\chris\anaconda3\envs\audio_app\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ════════════════════════════════════════════════════════════
# Function to Fuse Bass, Drums, and Other Stems
# ════════════════════════════════════════════════════════════
def fuse_audio_stems(bass_file, drums_file, other_file, output_format="mp3"):
    """
    Fuse the bass, drums, and other stems into one audio file without vocals.
    
    Args:
        bass_file (str): Path to the bass stem audio file.
        drums_file (str): Path to the drums stem audio file.
        other_file (str): Path to the other stem audio file.
        output_format (str): Desired output format, e.g., 'mp3' or 'wav'.

    Returns:
        str: Path to the fused audio file.
    """
    if not bass_file or not drums_file or not other_file:
        return None

    # Load the audio stems
    bass = AudioSegment.from_file(bass_file)
    drums = AudioSegment.from_file(drums_file)
    other = AudioSegment.from_file(other_file)

    # Fuse the audio stems by overlaying them
    fused_audio = bass.overlay(drums).overlay(other)

    # Export the fused audio to a temporary file
    output_path = f"fused_audio.{output_format}"
    fused_audio.export(output_path, format=output_format)

    return output_path

# ════════════════════════════════════════════════════════════
# Update Gradio Interface for Lyric Extraction and Stem Fusion
# ════════════════════════════════════════════════════════════
def create_lyrics_interface():
    with gr.Blocks() as interface:
        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### Audio Input")
                audio_input = gr.Audio(type="filepath", label="Upload Audio File", sources="upload")
                extract_button = gr.Button("Extract Lyrics")

            with gr.Column(scale=1):
                gr.Markdown("### Lyrics Output")
                lyrics_output = gr.Textbox(label="Lyrics")

                gr.Markdown("### Translate Lyrics")
                language_code = gr.Dropdown(choices=get_available_languages(), label="Language", interactive=True)
                translated_output = gr.Textbox(label="Translated Lyrics")
                translate_button = gr.Button("Translate")

        # New Row for Fusing Audio Stems
        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### Fuse Bass, Drums, and Other Stems")
                bass_input = gr.Audio(type="filepath", label="Bass Stem", sources="upload")
                drums_input = gr.Audio(type="filepath", label="Drums Stem", sources="upload")
                other_input = gr.Audio(type="filepath", label="Other Stem", sources="upload")
                fuse_button = gr.Button("Fuse Stems")

            with gr.Column(scale=1):
                gr.Markdown("### Fused Output")
                fused_output = gr.Audio(label="Fused Audio")

        # Link buttons to functions
        extract_button.click(
            process_audio_lyric_extraction,
            inputs=[audio_input],
            outputs=[lyrics_output],
        )

        translate_button.click(
            process_audio_lyric_translation,
            inputs=[lyrics_output, language_code],
            outputs=[translated_output],
        )

        fuse_button.click(
            fuse_audio_stems,
            inputs=[bass_input, drums_input, other_input],
            outputs=[fused_output],
        )

    return interface

# Render the Gradio interface
interface = create_lyrics_interface()

# Launch the Gradio interface
interface.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [5]:

from mido import MidiFile